# import

In [1]:
import mne
import numpy as np
import pickle
from scipy.signal import firwin, filtfilt
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
import logging
import warnings
import torch
import random
from scipy.io import loadmat
warnings.filterwarnings("ignore", category=RuntimeWarning, module="mne")
# Completely silence MNE-Python output
mne.set_log_level('WARNING')  # or 'ERROR' for even less output
logging.getLogger('mne').setLevel(logging.WARNING)
#mne.set_log_level('debug')

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

# preprocess

In [2]:
def process(file_path, clas):

    try:
        raw = mne.io.read_raw_edf(file_path, preload=True)
    except Exception as e:
        print(f"\nERROR during file loading: {e}")
        return

    epochs = mne.make_fixed_length_epochs(
        raw, 
        duration=10.0, 
        overlap=1.0, 
    )

    epochs.load_data() 
    label = epochs.get_data().shape[0]
    labels=None
    if clas == 'H':
        labels = [0]*label
    else:
        labels = [1] * label

    return {"epochs":epochs, "labels":labels}


# Data Loader

In [3]:
import glob
from torch.utils.data import Dataset, DataLoader
import scipy.io as sio
import os
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
from scipy.ndimage import gaussian_filter1d

def gauss_smooth(inputs, device, smooth_kernel_std=2, smooth_kernel_size=100,  padding='same'):

    #print(inputs.shape)
    inputs = inputs.transpose(0, 2, 1)
    # Get Gaussian kernel
    inp = np.zeros(smooth_kernel_size, dtype=np.float32)
    inp[smooth_kernel_size // 2] = 1
    gaussKernel = gaussian_filter1d(inp, smooth_kernel_std)
    validIdx = np.argwhere(gaussKernel > 0.01)
    gaussKernel = gaussKernel[validIdx]
    gaussKernel = np.squeeze(gaussKernel / np.sum(gaussKernel))

    # Convert to tensor
    gaussKernel = torch.tensor(gaussKernel, dtype=torch.float32, device=device)
    gaussKernel = gaussKernel.view(1, 1, -1)  # [1, 1, kernel_size]

    # Prepare convolution
    B, T, C = inputs.shape
    inputs = inputs.transpose(0, 2, 1)  # [B, C, T]
    inputs = torch.tensor(inputs, dtype=torch.float32, device=device)
    
    gaussKernel = gaussKernel.repeat(C, 1, 1)  # [C, 1, kernel_size]

    # Perform convolution
    smoothed = F.conv1d(inputs, gaussKernel, padding=padding, groups=C)
    return smoothed.detach().cpu().numpy()  # [B, T, C]

from scipy.signal import filtfilt, butter

def bandpass(data, low=1, high=40, fs=256, order=4):
    nyq = fs / 2
    b, a = butter(order, [low/nyq, high/nyq], btype='band')
    
    # Apply on axis=2 (time axis)
    return filtfilt(b, a, data, axis=2)

import pandas as pd

class BCI4_2a_Dataset(Dataset):

    def __init__(self, data_dir, transform=None, target_transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.target_transform = target_transform
        self.data, self.labels = self._load_data()

        self.labels = self.labels 
        # In your dataset __init__:
        print(f"Class distribution: {np.bincount(self.labels)}")
        print(f"Class ratio: {np.bincount(self.labels)[1] / len(self.labels):.2%}")


    def _load_data(self):

        df = pd.read_csv(self.data_dir, header=None)
        df = df[df.iloc[:, 2] != -1]
        labels = df.iloc[:, 2].values
        all_data = []

        for i, row in df.iterrows():
            data = row.iloc[788:].astype(float).values.reshape(64, 400)
            all_data.append(data)
            exp_data = np.expand_dims(data, axis=0)
            aug1 = gauss_smooth(exp_data, device='cpu', smooth_kernel_std=0.7, smooth_kernel_size=20)
            aug2 = gauss_smooth(exp_data, device='cpu', smooth_kernel_std=2.0, smooth_kernel_size=50)
            aug3 = gauss_smooth(exp_data, device='cpu', smooth_kernel_std=2, smooth_kernel_size=50)
            all_data.append(np.squeeze(aug1, axis=0))
            all_data.append(np.squeeze(aug2, axis=0))
            all_data.append(np.squeeze(aug3, axis=0))
        
        # Convert list → NumPy array
        all_data = np.stack(all_data)
 
        labels = np.repeat(labels, 3)

        return all_data, labels
    
    def __len__(self):

        return len(self.labels)
    
    def __getitem__(self, idx):
        x = self.data[idx]
        y = self.labels[idx]
        
        x = torch.from_numpy(x).float()  # shape: (1, timepoints, channels)
        
        if self.transform:
            x = self.transform(x)
        if self.target_transform:
            y = self.target_transform(y)
            
        return x, y

def get_data_loaders(data_dir, batch_size=32, test_split= 0.2, val_split=0.2, random_state=42):
    # Get all subjects except the test subject for training/validation
    
    #train_val_subjects = [s for s in all_subjects if s not in test_subject]
    
    # Create datasets
    train_val_dataset = BCI4_2a_Dataset(data_dir)
    single_trial_shape = train_val_dataset.data[0].shape
    # labels = np.array(train_val_dataset.labels)
    # class_weights = compute_class_weight(class_weight='balanced',
    #                                  classes=np.unique(labels),
    #                                  y=labels)

    # class_weights = torch.tensor(class_weights, dtype=torch.float32)
    print(f"Train+Val dataset size: {len(train_val_dataset)}")
    
    train_idx, val_idx = train_test_split(
        range(len(train_val_dataset)),
        test_size=test_split,          # e.g., 0.2
        random_state=random_state,
        stratify=train_val_dataset.labels
    )
  

    
    train_dataset = torch.utils.data.Subset(train_val_dataset, train_idx)
    val_dataset = torch.utils.data.Subset(train_val_dataset, val_idx)
    
    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    print(f"Train batches: {len(train_loader)}")
    print(f"Val batches: {len(val_loader)}")
    print(f"Single Trial Shape: {single_trial_shape}")
    return train_loader, val_loader, single_trial_shape,None# class_weights



# Model

In [4]:
import torch
import torch.nn as nn
import math

class ChannelAttention(nn.Module):
    def __init__(self, num_channels, reduction_ratio=4, dropout=0.1):
        super(ChannelAttention, self).__init__()
        self.num_channels = num_channels
        self.reduction_ratio = reduction_ratio

        self.mlp = nn.Sequential(
            nn.Linear(num_channels, num_channels // reduction_ratio),
            nn.ReLU(),
            nn.Dropout(dropout),  # Added dropout
            nn.Linear(num_channels // reduction_ratio, num_channels)
        )

    def forward(self, x):
        channel_avg = torch.mean(x, dim=2, keepdim=True)  
        channel_max, _ = torch.max(x, dim=2, keepdim=True) 
        combined = channel_avg + channel_max  
        combined = combined.squeeze(2)
        attention = self.mlp(combined)
        attention = torch.sigmoid(attention).unsqueeze(2)
        attended_x = x * attention
        return attended_x, attention.squeeze(2)  # Return proper attention scores


class LearnableSTFT(nn.Module):
    def __init__(self, window_size, hop_size, learnable_window=True, dropout=0.05):
        super(LearnableSTFT, self).__init__()
        
        self.window_size = window_size
        self.hop_size = hop_size
        self.dft_size = window_size
        self.learnable_window = learnable_window
        
        # Initialize with Hamming window
        initial_window = 0.54 - 0.46 * torch.cos(
            2 * math.pi * torch.arange(window_size, dtype=torch.float32) / (window_size - 1)
        )
        
        if learnable_window:
            self.window = nn.Parameter(initial_window)
        else:
            # Fixed window reduces overfitting
            self.register_buffer('window', initial_window)
        
        # Batch normalization after STFT
        self.batch_norm = nn.BatchNorm2d(1)
        self.dropout = nn.Dropout2d(dropout)
        
        dft_matrix = self._create_dft_matrix(self.dft_size, self.window_size)
        self.register_buffer('dft_matrix', dft_matrix)

    def _create_dft_matrix(self, dft_size, window_size):
        k = torch.arange(dft_size).unsqueeze(1)
        n = torch.arange(window_size)
        angle = -2 * math.pi * k * n / dft_size
        dft_matrix = torch.complex(torch.cos(angle), torch.sin(angle))
        return dft_matrix

    def forward(self, signal):
        if signal.dim() == 1:
            signal = signal.unsqueeze(0).unsqueeze(0)
        elif signal.dim() == 2:
            signal = signal.unsqueeze(1)
            
        batch_size, num_channels, num_samples = signal.shape
        signal_reshaped = signal.reshape(batch_size * num_channels, num_samples)
        
        frames = signal_reshaped.unfold(dimension=1, size=self.window_size, step=self.hop_size)
        num_frames_unfolded = frames.shape[1]
        expected_num_frames = int(math.ceil((num_samples - self.window_size) / self.hop_size)) + 1
        
        if num_frames_unfolded < expected_num_frames:
            padding_amount = (expected_num_frames - 1) * self.hop_size + self.window_size - num_samples
            padded_signal = torch.nn.functional.pad(signal_reshaped, (0, padding_amount))
            frames = padded_signal.unfold(1, self.window_size, self.hop_size)
        
        windowed_frames = frames * self.window
        BC, F, W = windowed_frames.shape
        windowed_frames_reshaped = windowed_frames.reshape(BC * F, W)
        
        windowed_frames_complex = windowed_frames_reshaped.to(self.dft_matrix.dtype)
        stft_result_reshaped = self.dft_matrix @ windowed_frames_complex.T
        stft_result = stft_result_reshaped.T.reshape(batch_size, num_channels, F, self.dft_size)
        
        # Apply magnitude and normalization
        stft_magnitude = torch.abs(stft_result)
        
        # Reshape for batch norm: (B*C, 1, F, freq_bins)
        stft_normalized = stft_magnitude.reshape(batch_size * num_channels, 1, F, self.dft_size)
        stft_normalized = self.batch_norm(stft_normalized)
        stft_normalized = self.dropout(stft_normalized)
        stft_normalized = stft_normalized.reshape(batch_size, num_channels, F, self.dft_size)
        
        return stft_normalized


class Attention4D(nn.Module):
    def __init__(self, in_channels, time_frames, freq_bins, d_model=128, n_heads=4, 
                 d_ff=256, dropout=0.2, num_layers=1):
        super().__init__()
        
        self.d_model = d_model
        self.time_frames = time_frames
        self.num_layers = num_layers
        
        in_features = in_channels * freq_bins
        
        # Project input to d_model with layer norm
        self.projection = nn.Sequential(
            nn.Linear(in_features, d_model),
            nn.LayerNorm(d_model),
            nn.Dropout(dropout)
        )
        
        # Learnable positional encoding with smaller init
        self.positional_encoding = nn.Parameter(
            torch.randn(1, time_frames, d_model) * 0.02
        )
        
        # Stack multiple transformer layers for better representation
        self.transformer_layers = nn.ModuleList([
            TransformerBlock(d_model, n_heads, d_ff, dropout)
            for _ in range(num_layers)
        ])
        
        self.final_norm = nn.LayerNorm(d_model)
        
    def forward(self, x):
        batch_size = x.shape[0]
        
        # Reshape: (B, C, T, F) -> (B, T, C*F)
        x = x.permute(0, 2, 1, 3)
        x = x.reshape(batch_size, self.time_frames, -1)
        
        # Project and add positional encoding
        x = self.projection(x)
        x = x + self.positional_encoding
        
        # Store attention weights from all layers
        all_attention_weights = []
        
        # Pass through transformer layers
        for layer in self.transformer_layers:
            x, attn_weights = layer(x)
            all_attention_weights.append(attn_weights)
        
        x = self.final_norm(x)
        
        return x, all_attention_weights


class TransformerBlock(nn.Module):
    def __init__(self, d_model, n_heads, d_ff, dropout):
        super().__init__()
        
        self.layernorm1 = nn.LayerNorm(d_model)
        self.attention = nn.MultiheadAttention(
            embed_dim=d_model, 
            num_heads=n_heads, 
            dropout=dropout, 
            batch_first=True
        )
        self.dropout1 = nn.Dropout(dropout)
        
        self.layernorm2 = nn.LayerNorm(d_model)
        self.feed_forward = nn.Sequential(
            nn.Linear(d_model, d_ff),
            nn.GELU(),  # GELU often works better than ReLU
            nn.Dropout(dropout),
            nn.Linear(d_ff, d_model)
        )
        self.dropout2 = nn.Dropout(dropout)
    
    def forward(self, x):
        # Pre-norm architecture (more stable)
        x_norm = self.layernorm1(x)
        attn_output, attn_weights = self.attention(x_norm, x_norm, x_norm, 
                                                    need_weights=True, 
                                                    average_attn_weights=True)
        x = x + self.dropout1(attn_output)
        
        x_norm = self.layernorm2(x)
        ff_output = self.feed_forward(x_norm)
        x = x + self.dropout2(ff_output)
        
        return x, attn_weights


class Classifier(nn.Module):
    def __init__(self, in_features, num_classes, dropout=0.3):
        super().__init__()
        
        self.pooling = nn.AdaptiveAvgPool1d(1)
        
        # Add intermediate layer for better capacity
        self.classifier = nn.Sequential(
            nn.Linear(in_features, in_features // 2),
            nn.LayerNorm(in_features // 2),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(in_features // 2, num_classes)
        )
    
    def forward(self, x):
        x = x.permute(0, 2, 1)  # (B, d_model, T)
        x = self.pooling(x).squeeze(2)  # (B, d_model)
        output = self.classifier(x)  # (B, num_classes)
        return output


class EEGClassifier(nn.Module):
    def __init__(self, num_samples, num_classes=4, window_percent=0.25, 
                 overlap_percent=0.10, d_model=128, n_heads=8, 
                 transformer_layers=1, dropout=0.2, learnable_window=True):

        super(EEGClassifier, self).__init__()
        
        window_size = int(num_samples * window_percent)
        hop_size = int(num_samples * overlap_percent)
        
        self.window_size = max(window_size, 40)
        self.hop_size = max(hop_size, 20)
        
        self.channel_attn = ChannelAttention(num_channels=64, dropout=dropout)
        
        self.learnable_stft = LearnableSTFT(
            window_size=self.window_size,
            hop_size=self.hop_size,
            learnable_window=learnable_window,
            dropout=dropout * 0.5
        )
        
        time_frames = int(math.ceil((num_samples - self.window_size) / self.hop_size)) + 1
        
        self.attention_module = Attention4D(
            in_channels=64,
            time_frames=time_frames,
            freq_bins=self.window_size,
            d_model=d_model,
            n_heads=n_heads,
            num_layers=transformer_layers,
            dropout=dropout
        )
        
        self.classifier = Classifier(
            in_features=d_model, 
            num_classes=num_classes,
            dropout=dropout
        )

    def forward(self, x, return_attention=False):

        # Channel attention
        x, channel_attn_scores = self.channel_attn(x)
        
        # STFT
        x = self.learnable_stft(x)
        
        # Transformer attention
        x, transformer_attn_weights = self.attention_module(x)
        
        # Classification
        logits = self.classifier(x)
        
        if return_attention:
            attention_dict = {
                'channel_attention': channel_attn_scores,  # (B, 22)
                'transformer_attention': transformer_attn_weights,  # List of (B, T, T) for each layer
            }
            return logits, attention_dict
        
        return logits


class AttentionRegularizedLoss(nn.Module):
    def __init__(self, num_classes, alpha=0.01, beta=0.01):
        super().__init__()
        self.ce_loss = nn.CrossEntropyLoss()
        self.alpha = alpha  # Channel attention regularization weight
        self.beta = beta    # Transformer attention regularization weight
        
    def forward(self, logits, targets, attention_dict):
        ce = self.ce_loss(logits, targets)
        channel_attn = attention_dict['channel_attention']  # (B, C)
        channel_attn_norm = channel_attn + 1e-8
        channel_attn_norm = channel_attn_norm / channel_attn_norm.sum(dim=1, keepdim=True)
        channel_entropy = -(channel_attn_norm * torch.log(channel_attn_norm)).sum(dim=1).mean()
        channel_reg = -self.alpha * channel_entropy  # Negative because we want to maximize entropy
        transformer_attn = attention_dict['transformer_attention'][-1]  # (B, T, T)
        attn_norm = transformer_attn + 1e-8
        attn_norm = attn_norm / attn_norm.sum(dim=-1, keepdim=True)
        attn_entropy = -(attn_norm * torch.log(attn_norm)).sum(dim=-1).mean()
        transformer_reg = -self.beta * attn_entropy  # Maximize entropy

        total_loss = ce + channel_reg + transformer_reg
        
        return total_loss, {
            'ce_loss': ce.item(),
            'channel_reg': channel_reg.item(),
            'transformer_reg': transformer_reg.item(),
            'total_loss': total_loss.item()
        }




# test model

In [5]:
# if __name__ == "__main__":
#     # Model initialization
#     model = EEGClassifier(
#         num_samples=1000,
#         num_classes=4,
#         d_model=64,  # Reduced from 128
#         n_heads=4,   # Reduced from 8
#         transformer_layers=2,
#         dropout=0.3,
#         learnable_window=False  # Start with fixed window
#     )
    
#     # Loss function
#     criterion = AttentionRegularizedLoss(num_classes=4, alpha=0.01, beta=0.01)
    
#     # Dummy data
#     x = torch.randn(4, 22, 1000)  # (batch, channels, time)
#     y = torch.randint(0, 4, (4,))
    
#     # Forward pass with attention
#     logits, attention_dict = model(x, return_attention=True)
    
#     # Calculate loss
#     loss, loss_dict = criterion(logits, y, attention_dict)
    
#     print(f"Logits shape: {logits.shape}")
#     print(f"Channel attention shape: {attention_dict['channel_attention'].shape}")
#     print(f"Num transformer layers: {len(attention_dict['transformer_attention'])}")
#     print(f"Transformer attention shape: {attention_dict['transformer_attention'][0].shape}")
#     print(f"Total loss: {loss.item():.4f}")
#     print(f"Loss components: {loss_dict}")

# Train model

In [6]:
import time
from torch.utils.tensorboard import SummaryWriter
from torch.optim.lr_scheduler import ReduceLROnPlateau
import json
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

class EEGTrainer:
    def __init__(self, model, train_loader, val_loader ,class_weights=None,test_loader=None, config=None):

        self.model = model
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.test_loader = test_loader
        
        # Set device
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = self.model.to(self.device)
        self.class_weights = class_weights
        # Default configuration
        self.config = {
            'lr': 1e-3,
            'weight_decay': 1e-4,
            'patience': 10,
            'min_lr': 1e-6,
            'epochs': 100,
            'save_dir': 'experiments',
            'experiment_name': f'exp_{time.strftime("%Y%m%d-%H%M%S")}',
            'save_attention_maps': True,
            'attention_map_freq': 5
        }
        
        # Update with user config if provided
        if config:
            self.config.update(config)
            
        # Create experiment directory
        self.exp_dir = os.path.join(self.config['save_dir'], self.config['experiment_name'])
        os.makedirs(self.exp_dir, exist_ok=True)
        
        # Initialize components
        self._init_components()
        
        # Save config
        self._save_config()
        
    def _init_components(self):
        """Initialize training components"""
        # Loss function (CrossEntropy + KLDiv for attention regularization)
        #self.criterion = nn.CrossEntropyLoss()#weight=self.class_weights)
        self.criterion = AttentionRegularizedLoss(num_classes=10)
        # Optimizer
        self.optimizer = torch.optim.AdamW(
            self.model.parameters(),
            lr=self.config['lr'],
            weight_decay=self.config['weight_decay']
        )
        
        # Learning rate scheduler
        self.scheduler = ReduceLROnPlateau(
            self.optimizer,
            mode='max',
            factor=0.5,
            patience=self.config['patience']//2,
            min_lr=self.config['min_lr'],
            #verbose=True
        )
        
        self.best_val_acc = 0.0
        self.early_stop_counter = 0
        
        # Tensorboard writer
        self.writer = SummaryWriter(log_dir=self.exp_dir)
        
        # Attention maps directory
        if self.config['save_attention_maps']:
            self.attention_dir = os.path.join(self.exp_dir, 'attention_maps')
            os.makedirs(self.attention_dir, exist_ok=True)
    
    def _save_config(self):
        config_path = os.path.join(self.exp_dir, 'config.json')
        with open(config_path, 'w') as f:
            json.dump(self.config, f, indent=4)
    
    def _compute_metrics(self, outputs, labels):
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        accuracy = correct / labels.size(0)
        return accuracy
    
    def _log_metrics(self, phase, metrics, epoch):
        loss = metrics['loss']
        acc = metrics['accuracy']
        pre = metrics['precision']
        rec = metrics['recall']
        f1 = metrics['f1']
        
        # Console logging
        print(f"{phase.capitalize()} - Epoch: {epoch+1} | "
              f"Loss: {loss:.4f} | Acc: {acc:.2%} | Precision: {pre:.2%} | recall: {rec:.2%} | f1: {f1:.2%}")
        
        # Tensorboard logging
        self.writer.add_scalar(f'Loss/{phase}', loss, epoch)
        self.writer.add_scalar(f'Accuracy/{phase}', acc, epoch)
        
        
        # Log learning rate
        if phase == 'train':
            lr = self.optimizer.param_groups[0]['lr']
            self.writer.add_scalar('LR', lr, epoch)
    
    def _save_checkpoint(self, epoch, is_best=False):
        """Save model checkpoint"""
        state = {
            'epoch': epoch,
            'state_dict': self.model.state_dict(),
            'optimizer': self.optimizer.state_dict(),
            'scheduler': self.scheduler.state_dict(),
            'best_val_acc': self.best_val_acc
        }
        
        # Save regular checkpoint
        checkpoint_path = os.path.join(self.exp_dir, f'checkpoint_epoch_{epoch}.pth')
        torch.save(state, checkpoint_path)
        
        # Save best model
        if is_best:
            best_path = os.path.join(self.exp_dir, 'best_model.pth')
            torch.save(state, best_path)
    

    


    def train_epoch(self, epoch):
        """Train for one epoch"""
        self.model.train()
        running_loss = 0.0
        total_samples = 0

        # Store all predictions and labels for epoch-wise metrics
        all_preds = []
        all_labels = []

        for inputs, labels in self.train_loader:
            inputs = inputs.to(self.device)
            labels = labels.to(self.device)

            # Zero gradients
            self.optimizer.zero_grad()
            
            # Forward pass
            outputs, attention_dict = self.model(inputs, return_attention=True)
            loss ,_ = self.criterion(outputs, labels,attention_dict)
            
            # Backward pass and optimize
            loss.backward()
            self.optimizer.step()
            
            # ----- accumulate loss -----
            batch_size = inputs.size(0)
            running_loss += loss.item() * batch_size
            total_samples += batch_size

            # ----- accumulate preds + labels for metrics -----
            preds = torch.argmax(outputs, dim=1)          # [B]
            all_preds.append(preds.detach().cpu())
            all_labels.append(labels.detach().cpu())

        # ---- end of epoch: stack everything and compute metrics ----
        all_preds = torch.cat(all_preds).numpy()
        all_labels = torch.cat(all_labels).numpy()

        epoch_loss = running_loss / total_samples

        # accuracy
        epoch_acc = accuracy_score(all_labels, all_preds)

        # precision, recall, f1
        precision, recall, f1, _ = precision_recall_fscore_support(
            all_labels,
            all_preds,
            average='weighted',      # change to 'macro' / 'binary' if needed
            zero_division=0          # avoid NaN if a class is missing in preds
        )

        return {
            'loss': epoch_loss,
            'accuracy': epoch_acc,
            'precision': precision,
            'recall': recall,
            'f1': f1,
        }


    
    def validate_epoch(self, epoch):
        """Validate for one epoch"""
        self.model.eval()
        running_loss = 0.0
        total_samples = 0

        all_preds = []
        all_labels = []
        
        with torch.no_grad():
            for inputs, labels in self.val_loader:
                inputs = inputs.to(self.device)
                labels = labels.to(self.device)
     
                # Forward pass
                #outputs = self.model(inputs)
                outputs, attention_dict = self.model(inputs, return_attention=True)
                loss, _ = self.criterion(outputs, labels, attention_dict)
                #loss = self.criterion(outputs, labels)
                
                batch_size = inputs.size(0)
                running_loss += loss.item() * batch_size
                total_samples += batch_size

                # ----- accumulate preds + labels for metrics -----
                preds = torch.argmax(outputs, dim=1)          # [B]
                all_preds.append(preds.detach().cpu())
                all_labels.append(labels.detach().cpu())

            # ---- end of epoch: stack everything and compute metrics ----
            all_preds = torch.cat(all_preds).numpy()
            all_labels = torch.cat(all_labels).numpy()

            epoch_loss = running_loss / total_samples

            # accuracy
            epoch_acc = accuracy_score(all_labels, all_preds)

            # precision, recall, f1
            precision, recall, f1, _ = precision_recall_fscore_support(
                all_labels,
                all_preds,
                average='weighted',      # change to 'macro' / 'binary' if needed
                zero_division=0          # avoid NaN if a class is missing in preds
            )

            return {
                'loss': epoch_loss,
                'accuracy': epoch_acc,
                'precision': precision,
                'recall': recall,
                'f1': f1,
            }

    
    def train(self):
        """Main training loop"""
        start_time = time.time()
        
        for epoch in range(self.config['epochs']):
            # Train and validate
            train_metrics = self.train_epoch(epoch)
            val_metrics = self.validate_epoch(epoch)
            
            # Log metrics
            self._log_metrics('train', train_metrics, epoch)
            self._log_metrics('val', val_metrics, epoch)
            
            # Save attention maps periodically
            
            # Step scheduler
            self.scheduler.step(val_metrics['accuracy'])
            
            # Check for best model
            if val_metrics['accuracy'] > self.best_val_acc:
                self.best_val_acc = val_metrics['accuracy']
                self._save_checkpoint(epoch, is_best=True)
                self.early_stop_counter = 0
            else:
                self.early_stop_counter += 1
            
            # Save checkpoint periodically
            if epoch % 10 == 0:
                self._save_checkpoint(epoch)
            
            # Early stopping
            # if self.early_stop_counter >= self.config['patience']:
            #     print(f"Early stopping at epoch {epoch+1}")
            #     break
        
        # Training complete
        training_time = time.time() - start_time
        print(f"Training completed in {training_time//60:.0f}m {training_time%60:.0f}s")
        print(f"Best validation accuracy: {self.best_val_acc:.2%}")
        
        # Test if test loader provided
        if self.test_loader:
            test_acc = self.test()
            print(f"Test accuracy: {test_acc:.2%}")
        
        # Close tensorboard writer
        self.writer.close()
        
        return self.best_val_acc
    
    def test(self):
        """Evaluate on test set"""
        self.model.eval()
        running_acc = 0.0
        total_samples = 0
        
        # Load best model
        best_path = os.path.join(self.exp_dir, 'best_model.pth')
        if os.path.exists(best_path):
            checkpoint = torch.load(best_path)
            self.model.load_state_dict(checkpoint['state_dict'])
            print(f"Loaded best model from epoch {checkpoint['epoch']}")
        
        all_preds=[]
        all_labels=[]
        with torch.no_grad():
            for inputs, labels in self.test_loader:
                inputs = inputs.to(self.device)
                labels = labels.to(self.device)
                
                outputs = self.model(inputs)
                batch_size = inputs.size(0)
                total_samples += batch_size

                # ----- accumulate preds + labels for metrics -----
                preds = torch.argmax(outputs, dim=1)          # [B]
                all_preds.append(preds.detach().cpu())
                all_labels.append(labels.detach().cpu())

            # ---- end of epoch: stack everything and compute metrics ----
            all_preds = torch.cat(all_preds).numpy()
            all_labels = torch.cat(all_labels).numpy()

            epoch_acc = accuracy_score(all_labels, all_preds)

            # precision, recall, f1
            precision, recall, f1, _ = precision_recall_fscore_support(
                all_labels,
                all_preds,
                average='binary',      # change to 'macro' / 'binary' if needed
                zero_division=0          # avoid NaN if a class is missing in preds
            )
        
        test_acc = epoch_acc
        self.writer.add_scalar('Accuracy/test', test_acc)
        print("acc: ",test_acc)
        print("precision:", precision)
        print("recall:", recall)
        print("F1 :", f1)
        return epoch_acc





2025-11-30 03:53:07.478214: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-30 03:53:07.792906: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-11-30 03:53:07.792971: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-11-30 03:53:07.795076: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-11-30 03:53:07.993711: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: A

# Main Code

In [7]:
if __name__ == "__main__":
    # Example configuration
    config = {
        'lr': 3e-5,
        'weight_decay': 1e-5,
        'patience': 15,
        'epochs': 110,
        'experiment_name': 'eeg_attention_experiment',
        'save_attention_maps': True
    }
    
    
    
    train_loader, val_loader, trail_shape, class_weights = get_data_loaders(data_dir="/teamspace/studios/this_studio/mnist_data/MindBigData64_Mnist2022-EEGv0.016.txt")
    model = EEGClassifier(num_samples = trail_shape[-1], num_classes=10)
    # Create trainer
    trainer = EEGTrainer(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        class_weights = class_weights,
        config=config
    )
    
    # Start training
    best_val_acc = trainer.train()

Class distribution: [342 417 345 333 363 312 321 414 306 345]
Class ratio: 11.92%
Train+Val dataset size: 3498
Train batches: 88
Val batches: 22
Single Trial Shape: (64, 400)
Train - Epoch: 1 | Loss: 2.2678 | Acc: 11.01% | Precision: 10.60% | recall: 11.01% | f1: 8.96%
Val - Epoch: 1 | Loss: 2.2404 | Acc: 12.00% | Precision: 4.42% | recall: 12.00% | f1: 3.06%
Train - Epoch: 2 | Loss: 2.2447 | Acc: 11.72% | Precision: 10.62% | recall: 11.72% | f1: 9.18%
Val - Epoch: 2 | Loss: 2.2345 | Acc: 13.29% | Precision: 4.60% | recall: 13.29% | f1: 6.13%
Train - Epoch: 3 | Loss: 2.2405 | Acc: 11.65% | Precision: 9.92% | recall: 11.65% | f1: 8.76%
Val - Epoch: 3 | Loss: 2.2334 | Acc: 12.29% | Precision: 3.98% | recall: 12.29% | f1: 5.92%
Train - Epoch: 4 | Loss: 2.2409 | Acc: 11.40% | Precision: 11.50% | recall: 11.40% | f1: 9.06%
Val - Epoch: 4 | Loss: 2.2353 | Acc: 12.14% | Precision: 3.00% | recall: 12.14% | f1: 3.85%
Train - Epoch: 5 | Loss: 2.2438 | Acc: 12.51% | Precision: 11.64% | recall: 12

In [8]:

# file_path = "/teamspace/studios/this_studio/mnist_s/MindBigData64_Mnist2022-EEGv0.016.txt"
# df = pd.read_csv(file_path, header=None)
# df = df[df.iloc[:, 2] != -1]

# df.columns = ['id', 'event', 'device', 'channel', 'code', 'size', 'data']
# df['data'] = df['data'].apply(lambda x: np.array(x.split(','), dtype=float))
# num_groups = len(df) // 14
# grouped = np.zeros((num_groups - 8, 14, 244), dtype=np.float32)
# group_idx = 0
# temp = None
# temp1=[]
# labels = []
# ind = 0
# for row, num in zip(df['data'], df['code']):

#     if num != -1:
#         if len(row) >= 244:
#             temp = df['data'].iloc[ind:ind+14]
#             ind+=14
#         else:
#             continue
        
#         if not temp.empty:
#             if all(len(i)>=244 for i in temp):
#                 for row in temp:
#                     temp1.append(row[:244])
#             else:
#                 continue
#         else:
#             continue


#         if len(temp1) == 14:
#             grouped[group_idx] = np.stack(temp1)
#             labels.append(num)
#             temp1=[]
#             temp = None
#             group_idx += 1


In [9]:

# labels = df.iloc[:, 2].values
# all_data = []

# for i, row in df.iterrows():
#     data = row.iloc[788:].astype(float).values.reshape(64, 400)
#     all_data.append(data)

# # Convert list → NumPy array
# all_data = np.stack(all_data)

    

